# MOD16A2.006: Terra Net Evapotranspiration 8-Day Global 500m

https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD16A2

## PART I: QUERY DATA FROM GEE SERVER

In [1]:
#installing dependencies
import ee
import geemap
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import seaborn as sns
import matplotlib as mpl
from pandas_profiling import ProfileReport

In [2]:
#user select raw data destination folder
destination_folder = input('destination_folder: ')

destination_folder: D:\Users\aster\Desktop\dzud-types\hoof-dzud\ET\ET-repo


In [5]:
#plot wold map
Map = geemap.Map()

In [7]:
#upload shapefile of AOE
prefectures_shp = '../ET-repo/prefectures-v3.shp'
prefectures = geemap.shp_to_ee(prefectures_shp)
Map.addLayer(prefectures, {}, 'prefectures')
Map

Map(bottom=6095.399993896484, center=[46.507466848985956, 100.61397975549423], controls=(WidgetControl(options…

### Input DATE, GEE Snippet and band. Can also modify data bounds and map colors. 

In [9]:
#user input start year 
#end year automatically set to most current data

start_date = str('2020-06-01')
end_date = str('2020-09-01')
    
dataset = ee.ImageCollection('MODIS/006/MOD16A2')     .filterDate (start_date, end_date)     .map(lambda image: image.clip(prefectures)) 
et = dataset.select('ET')    
etVis = {
    'min': 0.0,
    'max': 300.0,
    'palette': ['ffffff', 'fcd163', '99b718', '66a000', '3e8601', '207401', '056201',
    '004c00', '011301']    }
Map.setCenter(106, 47, 4)
Map.addLayer(et, etVis, 'ET-summer')
Map

Map(bottom=3149.2000122070312, center=[47, 106], controls=(WidgetControl(options=['position', 'transparent_bg'…

### Monthly Mean. Input desired variables.

In [11]:
summer_et = et.reduce(ee.Reducer.mean())
Map.addLayer(summer_et, etVis, 'summer_et_mean')
Map

Map(bottom=3182.0, center=[46.98025235521883, 105.99609375000001], controls=(WidgetControl(options=['position'…

In [12]:
summer_et

In [13]:
summer_et.getInfo()

{'type': 'Image',
 'bands': [{'id': 'ET_mean',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -32768,
    'max': 32767},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

### to TIF

##### Change file name:

In [15]:
feature = prefectures
roi = feature.geometry()
out_dir = os.path.join(os.path.expanduser('~'), destination_folder)
filename = os.path.join(out_dir, 'geemap_average_MODIS_ET_summer2020.tif')
image = monthly_et
geemap.ee_export_image(image, filename=filename, scale=1000, region=roi)

Generating URL ...
Please wait ...
Data downloaded to D:\Users\aster\Desktop\dzud-types\hoof-dzud\ET\ET-repo\geemap_average_MODIS_ET_summer2020.tif


### CONT. TO NOTEBOOK 2. TIF to CSV

zonal statistics by province csv table

In [16]:
out_dir = os.path.join(os.path.expanduser('~'), destination_folder)
out_et_stats = os.path.join(out_dir, 'et_stats_summer2020.csv')  

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM
geemap.zonal_statistics(summer_et, prefectures, out_et_stats, statistics_type='MEAN', scale=1000)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to D:\Users\aster\Desktop\dzud-types\hoof-dzud\ET\ET-repo\et_stats_summer2020.csv


##### THE END 👍🏽